### **Importación de librerías**

In [1]:
import pandas as pd
import numpy as np
import ast
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

### **Cargar el archivo csv, con el ETL realizado y dar formato de listas de python a las listas de las columnas**



In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies_credits_limpio.csv', parse_dates=['release_date'])

In [3]:
list_columns = ['genres', 'production_companies', 'production_countries', 'spoken_languages', 'cast']
df[list_columns] = df[list_columns].applymap(lambda x: list(ast.literal_eval(x)))

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45367 entries, 0 to 45366
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   budget                    45367 non-null  int64         
 1   genres                    45367 non-null  object        
 2   id                        45367 non-null  int64         
 3   original_language         45367 non-null  object        
 4   overview                  45367 non-null  object        
 5   popularity                45367 non-null  float64       
 6   production_companies      45367 non-null  object        
 7   production_countries      45367 non-null  object        
 8   release_date              45367 non-null  datetime64[ns]
 9   revenue                   45367 non-null  float64       
 10  runtime                   45367 non-null  float64       
 11  spoken_languages          45367 non-null  object        
 12  status            

### **Preparar los datos**

Función para quitar **stopwords**

In [5]:
# Descargar las stopwords en inglés si no están disponibles
nltk.download('stopwords')

# Obtener las stopwords en inglés
stop_words = set(stopwords.words('english'))
stop_words.update(',',';','!','?','.','(',')','$','#','+',':','...',' ','')

# Función para limpiar el texto
def clean_text(text):
    # Tokenizar el texto en palabras individuales
    tokens = text.split()

    # Eliminar las stopwords y los signos de puntuación
    tokens_cleaned = [token.lower() for token in tokens if token.lower() not in stop_words]

    # Unir las palabras limpias en un solo texto nuevamente
    text_cleaned = ' '.join(tokens_cleaned)

    return text_cleaned

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Modelo usando `TfidfVectorizer` para convertir texto en características numéricas

Combinar las columnas que se usarán y vectorizar el resultado de la combinación

In [6]:
# Combinar las columnas "overview", "genres", "cast" y "director" en una sola columna
df['combined'] = df['overview'] + ' ' + df['genres'].apply(lambda x: ' '.join(x)) + ' ' + df['cast'].apply(lambda x: ' '.join(''.join(name.split()) for name in x)) + \
    ' ' + df['director'].apply(lambda x: ''.join(x.split())) + ' ' + df['popularity'].astype(str) + ' ' + df['vote_average'].astype(str)

# Crear una instancia de TfidfVectorizer para vectorizar el texto combinado
tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1, 2))

# Aplicar TF-IDF al texto combinado
tfidf_matrix = tfidf.fit_transform(df['combined'])

Función que realiza la recomendación

In [7]:
def recomendacionTfidf(titulo):
    indices = df[df['title'] == titulo].index

    if len(indices) > 1:
        return {"error": f"Se encontraron múltiples películas con el título '{titulo}'."}
    elif len(indices) == 1:
        indice = indices[0]
        similitud_cos = cosine_similarity(tfidf_matrix[indice], tfidf_matrix).flatten()
        scores_similares = list(enumerate(similitud_cos))
        scores_similares = sorted(scores_similares, key=lambda x: x[1], reverse=True)
        indices_movie = [i[0] for i in scores_similares[1:6]]
        lista_recomendada = df['title'].iloc[indices_movie].tolist()
        respuesta = {
            'lista_recomendada': lista_recomendada
        }
        return respuesta
    else:
        return {"error": f"No se encontró ninguna película con el título '{titulo}'."}

Ejemplo

In [8]:
recomendacionTfidf('Avatar')

{'lista_recomendada': ['Avatar 2',
  'Stand by Me Doraemon',
  'The War of the Robots',
  'Guardians of the Galaxy Vol. 2',
  'A Trip to the Moon']}

### Modelo usando `CountVectorizer` para convertir texto en características numéricas

Combinar las columnas que se usarán y vectorizar el resultado de la combinación

In [9]:
# Seleccionar las columnas relevantes para la matriz de términos
df1 = df[['genres', 'overview', 'popularity', 'title', 'vote_average', 'release_year', 'cast', 'director']]

# Crear una instancia del vectorizador
vectorizer = CountVectorizer()

# Concatenar las columnas 'overview', 'genres', 'cast' y 'director' en un solo texto
textos_concatenados = df1['overview'].apply(clean_text) + ' ' + df1['genres'].apply(lambda x: ' '.join(x)) + ' ' + df1['cast'].apply(lambda x: ' '.join(''.join(name.split()) for name in x)) + \
  ' ' + df1['director'].apply(lambda x: ''.join(x.split())) + ' ' + df1['popularity'].astype(str) + ' ' + df1['vote_average'].astype(str)

# Crear la matriz de términos
terminos_mat = vectorizer.fit_transform(textos_concatenados)

Función que realiza la recomendación

In [10]:
def recomendacionCount(titulo: str):
    indices = df1[df1['title'] == titulo].index

    if len(indices) > 1:
        return {"error": f"Se encontraron múltiples películas con el título '{titulo}'."}
    elif len(indices) == 1:
        indice = indices[0]
        vector_pelicula = terminos_mat[indice]
        similitud_cos = cosine_similarity(vector_pelicula, terminos_mat)[0]
        scores_similares = list(enumerate(similitud_cos))
        scores_similares = sorted(scores_similares, key=lambda x: x[1], reverse=True)
        indices_movie = [i[0] for i in scores_similares[1:6]]
        lista_recomendada = df['title'].iloc[indices_movie].tolist()
        respuesta = {
            'lista_recomendada': lista_recomendada
        }
        return respuesta
    else:
        return {"error": f"No se encontró ninguna película con el título '{titulo}'."}

Ejemplo

In [11]:
recomendacionCount('Avatar')

{'lista_recomendada': ['Avatar 2',
  'A Trip to the Moon',
  'Fly Me to the Moon',
  "Frank Herbert's Dune",
  'Guardians of the Galaxy Vol. 2']}

### Modelo usando **KNN**

In [12]:
def recomendacionKNN(title):
    # Crear una instancia del clasificador KNN
    knn = NearestNeighbors(n_neighbors=6, metric='cosine')

    # Ajustar el modelo KNN utilizando la matriz tfidf_matrix
    knn.fit(tfidf_matrix)

    # Encontrar el índice de la película de entrada
    entrada_index = df[df['title'] == title].index[0]

    # Encontrar los vecinos más cercanos a la película de entrada
    distances, indices = knn.kneighbors(tfidf_matrix[entrada_index])

    # Recuperar los títulos de las películas recomendadas utilizando los índices encontrados
    recommended_movies = df.iloc[indices[0][1:]]['title']

    # Devolver las películas recomendadas
    return recommended_movies


In [13]:
recomendacionKNN('Avatar')

26532                          Avatar 2
28616              Stand by Me Doraemon
18001             The War of the Robots
26539    Guardians of the Galaxy Vol. 2
9954                 A Trip to the Moon
Name: title, dtype: object